In [14]:
import numpy as np
from qiskit import QuantumCircuit #, execute
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.circuit.library import PhaseEstimation, TwoLocal
from qiskit.circuit.library import HamiltonianGate
from qiskit.primitives import Estimator, Sampler
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import AerSimulator
from qiskit.compiler import transpiler

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
#from qiskit_nature.second_q.mappers import QubitConverter

from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
#from qiskit_nature.second_q.algorithms.initial_points import HFInitialPoint

In [15]:
# create hamiltonian for h2
# create initial problem to determine parameters for ansatz
dist = 0.735

atom_str = "H 0 0 0; H 0 0 " + str(dist)

print(atom_str)
driver = PySCFDriver(atom=atom_str, basis="sto-3g")
problem = driver.run()

num_spatial_orbitals = problem.num_spatial_orbitals
num_particles = problem.num_particles

print("Particles and orbitals for H2: ", num_particles, " ", num_spatial_orbitals)

H = problem.hamiltonian

H 0 0 0; H 0 0 0.735
Particles and orbitals for H2:  (1, 1)   2


In [16]:
estimator = Estimator()
optimizer = SLSQP()
ansatz = TwoLocal(rotation_blocks=['ry', 'rz'], entanglement_blocks='cz')
# create mapper
mapper = JordanWignerMapper()

fermionic_op = problem.hamiltonian.second_q_op()  # Get fermionic_op
    
qubit_op = mapper.map(fermionic_op)

vqe = VQE(estimator, ansatz, optimizer)

# set initialpoint to Harttreefock for problem
#initial_point = HFInitialPoint()
#initial_point.ansatz = ansatz
#initial_point.problem = problem
#vqe.initial_point = initial_point.to_numpy_array()

result = vqe.compute_minimum_eigenvalue(operator=qubit_op)
print(result.eigenvalue)

-1.8369674588961744


In [17]:
print(qubit_op)

SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IZII', 'ZIII', 'IIZZ', 'IZIZ', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.81054798+0.j,  0.17218393+0.j, -0.22575349+0.j,  0.17218393+0.j,
 -0.22575349+0.j,  0.12091263+0.j,  0.16892754+0.j,  0.16614543+0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j,
  0.16614543+0.j,  0.17464343+0.j,  0.12091263+0.j])


In [18]:
Hmat = qubit_op.to_matrix()

eig = np.linalg.eigvals(Hmat)
print(eig)

[-2.24911253e-01+0.j -1.85727503e+00+0.j -8.82722150e-01+0.j
 -1.24458455e+00+0.j  5.55111512e-16+0.j -1.25633907e+00+0.j
 -4.71896007e-01+0.j -1.24458455e+00+0.j -1.25633907e+00+0.j
 -1.16063174e+00+0.j -4.71896007e-01+0.j -3.53325104e-01+0.j
 -1.24458455e+00+0.j -1.16063174e+00+0.j -3.53325104e-01+0.j
  2.14278238e-01+0.j]


In [25]:
print(np.min(eig))

(-1.857275030202379+0j)


In [32]:
n_qpe_qbits = 10

U = HamiltonianGate(Hmat, 1, label='H')

# Obtain a solution via QPE
total_qbits = U.num_qubits + n_qpe_qbits
measure_circ = QuantumCircuit(total_qbits, n_qpe_qbits)
# set initial state to HF_State of H2: [1, 1, 0, 0]
measure_circ.x(n_qpe_qbits)
measure_circ.x(n_qpe_qbits+1)
qpe = PhaseEstimation(n_qpe_qbits, U)

measure_circ = measure_circ.compose(qpe)
measure_circ.measure(range(n_qpe_qbits), range(n_qpe_qbits))
print(measure_circ.decompose())


          ┌───┐                                                          »
 q_0: ────┤ H ├────────────■─────────────────────────────────────────────»
          ├───┤            │                                             »
 q_1: ────┤ H ├────────────┼─────────────────■───────────────────────────»
          ├───┤            │                 │                           »
 q_2: ────┤ H ├────────────┼─────────────────┼─────────────────■─────────»
          ├───┤            │                 │                 │         »
 q_3: ────┤ H ├────────────┼─────────────────┼─────────────────┼─────────»
          ├───┤            │                 │                 │         »
 q_4: ────┤ H ├────────────┼─────────────────┼─────────────────┼─────────»
          ├───┤            │                 │                 │         »
 q_5: ────┤ H ├────────────┼─────────────────┼─────────────────┼─────────»
          ├───┤            │                 │                 │         »
 q_6: ────┤ H ├──────────

In [33]:
from qiskit_aer.primitives import Sampler as AerSampler # import change!
 
sampler = AerSampler(run_options= {"method": "statevector"})
 
backend = AerSimulator(method='statevector', shots=2048)
job = sampler.run(measure_circ)

In [34]:
result = job.result()

In [35]:
print(result.quasi_dists)

[{44: 0.001953125, 500: 0.0009765625, 588: 0.005859375, 652: 0.0009765625, 460: 0.00390625, 908: 0.0009765625, 204: 0.01953125, 76: 0.0029296875, 716: 0.00390625, 396: 0.0009765625, 332: 0.02734375, 844: 0.9287109375, 812: 0.001953125}]


In [36]:
tmp_count = max(result.quasi_dists[0].items(), key=lambda x: x[1])
print(f'MAX count: {tmp_count}')
max_count = tmp_count[1]*2048


theta = max_count / 2**n_qpe_qbits
print(f'Theta value: {theta}')
eigenvalue = np.exp(2*1j*np.pi * theta)
print(f'QPE-approximated H-eigenvalue: {eigenvalue}')

MAX count: (844, 0.9287109375)
Theta value: 1.857421875
QPE-approximated H-eigenvalue: (0.624859488142386-0.7807372285720948j)
